In [14]:
# LLM Agents in Google ADK: Packing List & Itinerary Example
# Welcome 👋🏻 In this notebook, you'll install Google’s Agent Development Kit (ADK), 
# configure your environment, define two LLM agents, and run your agent interactions—all in just a few steps.
# You'll see how LLM Agents power real-world travel apps like WanderWise, using natural language understanding, 
# reasoning, and tool integration to deliver smart, actionable results.

In [31]:
# Install Google ADK for Python
# This is the foundational package that provides all the necessary components for building and running your agents.
# The --quiet flag suppresses verbose output during installation.
%pip install google-adk --quiet

Note: you may need to restart the kernel to use updated packages.


In [32]:
# Verify ADK Installation (Optional but Recommended)
%pip show google-adk

Name: google-adk
Version: 1.12.0
Summary: Agent Development Kit
Home-page: https://google.github.io/adk-docs/
Author: 
Author-email: Google LLC <googleapis-packages@google.com>
License: 
Location: c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\site-packages
Requires: absolufy-imports, anyio, authlib, click, fastapi, google-api-python-client, google-cloud-aiplatform, google-cloud-bigtable, google-cloud-secret-manager, google-cloud-spanner, google-cloud-speech, google-cloud-storage, google-genai, graphviz, mcp, opentelemetry-api, opentelemetry-exporter-gcp-trace, opentelemetry-sdk, pydantic, python-dateutil, python-dotenv, PyYAML, requests, sqlalchemy, starlette, tenacity, typing-extensions, tzlocal, uvicorn, watchdog, websockets
Required-by: 
Note: you may need to restart the kernel to use updated packages.


In [33]:
# Configure environment
import os

# Set GOOGLE_GENAI_USE_VERTEXAI to "False" to use the public Gemini API directly,
# rather than routing through Google Cloud's Vertex AI. This simplifies setup for quick demos.
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "False"

# Define the specific Gemini model we'll use. 
# 'gemini-2.0-flash' is a fast and efficient model.
MODEL_GEMINI_2_0_FLASH = "gemini-2.0-flash"

In [34]:
# Import Necessary Modules

# Agent: The fundamental class for creating an AI agent within ADK.
from google.adk.agents import Agent

# Runner: The orchestrator that manages the flow of messages and agent execution.
from google.adk.runners import Runner

# InMemorySessionService: A simple, in-memory service for managing conversation sessions. 
# Sessions are vital for mantaining context in multi-turn dialogues.
from google.adk.sessions import InMemorySessionService

# types from google.genai: Provides data structures (like Content and Part) to represent messages exchanged with the language model.
from google.genai import types

from IPython.display import Markdown, display

In [35]:
# Modular Function for Agent Interaction
async def run_adk_agent_interaction(
    agent: Agent,
    user_id: str,
    session_id: str,
    input_text: str,
    app_name: str = "default_app",
    session_service: InMemorySessionService = None,
) -> str:
    """
    Runs a single interaction with an ADK agent and returns its final text response.

    Args:
        agent: The ADK Agent instance to interact with.
        user_id: A unique identifier for the user initiating the interaction.
        session_id: A unique identifier for the conversation session.
                    Using a new ID for each distinct example ensures isolation.
        input_text: The textual message from the user.
        app_name: The name of the application using the agent. Defaults to "default_app".
        session_service: An optional InMemorySessionService instance. If None, a new
                         instance is created, making each call self-contained.

    Returns:
        The final text response from the agent as a string. Returns
        "No final response from agent." if no final response event is found.
    """
    # Use the provided session service if available, otherwise create a new one.
    # This flexibility allows for both isolated examples and continuous conversations.
    if session_service is None:
        session_service = InMemorySessionService()

    # Create a new session. In a multi-turn conversation, you would reuse
    # the same session_id across turns to maintain context. 
    # For these distinct examples, we'll use unique session_ids.
    await session_service.create_session(app_name=app_name, user_id=user_id, session_id=session_id)

    # Format the user's input into a Content object, which is how messages
    # are structured for the agent.
    content = types.Content(role="user", parts=[types.Part(text=input_text)])

    # Initialize the Runner: This is the engine that connects the user's message
    # to the agent and manages the session state.
    runner = Runner(agent=agent, app_name=app_name, session_service=session_service)

    # Run the agent: This sends the user's message to the agent for processing.
    # The 'runner.run()' method returns an iterable of events that describe
    # the agent's actions and responses.
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)

    # Process the events to find and extract the final response from the agent.
    for event in events:
        if event.is_final_response():
            # Concatenate all parts in the final response
            response = ''.join(part.text for part in event.content.parts if part.text)
            return response
    return "No final response from agent."

print("Modular function `run_adk_agent_interaction` successfully defined.")

# # NOTE: More details about Runners, Sessions, and advanced event processing are coming up in later tutorials—stay tuned!

Modular function `run_adk_agent_interaction` successfully defined.


In [36]:
# Define the Packing List Agent
# Our first agent, the packing_list_agent, is designed to generate a comprehensive packing list based on a user's travel details. 
# The key here is its instruction set, which guides the agent to ask clarifying questions and organize its output in a clear, categorized Markdown format.
# name: A unique identifier for the agent.
# model: The specific Gemini model (gemini-2.0-flash) powering the agent's reasoning.
# description: A brief summary of the agent's function.
# instruction: The detailed prompt that defines the agent's behavior, including what information to seek and how to format the output.
packing_list_agent = Agent(
    name="packing_list_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that generates a packing list for a trip given a destination, duration, and user preferences.",
    instruction="""
        You are a helpful travel assistant.
        When the user provides a destination and trip duration, generate a detailed packing list tailored for that location and timeframe.
        Ask about planned activities and any special preferences or needs (such as business, hiking, or traveling with children).
        If the user wishes, you may ask about personal preferences (such as specific clothing or toiletries), but do not assume based on gender.
        Make sure your suggestions are practical and organized by category (clothing, toiletries, electronics, documents, etc.).
        If the user provides additional details, refine the list accordingly.
        Present the packing list in a clear and easy-to-read markdown format.
    """,
)
# NOTE: In future lessons, you will integrate external tools (e.g., weather APIs) for even more personalized lists.
print("`packing_list_agent` successfully defined.")

`packing_list_agent` successfully defined.


In [37]:
#  Define common parameters for our interactions
APP_NAME = "wanderwise_app"
USER_ID = "user_001"

# Create a common session service instance to reuse.
common_session_service = InMemorySessionService()

# Define the user's input for the packing list agent.
PACKING_LIST_USER_INPUT = "I’m a 21 yo female traveling to Banff for 5 days in July. I plan to do some easy hiking and some city sightseeing."
PACKING_LIST_SESSION_ID = f"{APP_NAME}_packing_list_session_001" # Unique session ID for this interaction

print("--- Running Packing List Agent ---")

# Invoke the packing list agent using our modular helper function.
packing_list_response = await run_adk_agent_interaction(
    agent=packing_list_agent,
    user_id=USER_ID,
    session_id=PACKING_LIST_SESSION_ID,
    input_text=PACKING_LIST_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown for a nicely formatted list.
display(Markdown(f"{packing_list_response}"))

print("--- Packing List Agent Interaction Complete ---")

Exception in thread Thread-19 (_asyncio_thread_main):
Traceback (most recent call last):
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\D5DR2B7\AppData\Roaming\Python\Python313\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\adk\runners.py", line 165, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ~~~~~~~~~~^^^^^^
  File "c:

--- Running Packing List Agent ---


No final response from agent.

--- Packing List Agent Interaction Complete ---


In [22]:
# Congratulations 🎉  You’ve just defined and successfully run your first LLM agent for a real-world task: generating a personalized packing list. 
# This demonstrates how easily you can instruct an LLM to perform structured tasks and manage user interactions within the ADK framework.

In [23]:
# Itinerary Agent with Google Search Tool
# Now, let's go a step further: create an Itinerary Agent that uses the google_search tool! 
# This agent will be capable of generating a personalized itinerary for your trip, 
# leveraging live search results to find up-to-date recommendations for attractions, restaurants, and activities.

In [24]:
# Import the Google Search tool from ADK.
# This tool provides the agent with the ability to perform web searches.
from google.adk.tools import google_search

print("`google_search` tool imported.")

`google_search` tool imported.


In [25]:
# Define the Itinerary Agent
# The agent's instruction should explicitly guide it on when and how to use these tools. 
# Notice the instruction use the [google_search] tool. 
# This syntax helps the LLM understand that it has access to a tool named google_search and encourages it to use it appropriately.
itinerary_agent = Agent(
    name="itinerary_agent",
    model=MODEL_GEMINI_2_0_FLASH,
    description="An agent that creates a travel itinerary for a given destination and trip duration, using Google Search for up-to-date recommendations.",
    instruction="""
        You are a helpful and creative travel itinerary planner.
        When the user provides a destination and trip duration, use the [google_search] tool to find current and popular attractions, restaurants, and activities for each day.
        For each day, create a detailed schedule with 2-4 activities, including at least one meal suggestion and one local attraction, prioritizing the user's interests (such as art and food).
        Present the itinerary in a clear, easy-to-read markdown format, organized by day.

        Example format:

        ### Day 1
        - Morning: [Attraction or activity]
        - Lunch: [Restaurant or food experience]
        - Afternoon: [Attraction or activity]
        - Evening: [Dinner suggestion or event]

        ### Day 2
        ...

        Always use the [google_search] tool to find the latest recommendations for each activity or restaurant.
        When providing recommendations, if a direct and relevant official website or information page URL is prominently found in the search results for a specific item, include it as a Markdown hyperlink like [Item Name](URL).
        Be concise, friendly, and ensure the itinerary is complete for all days requested.
    """,
    tools=[google_search], # This is the crucial part: assigning the tool to the agent!
)

print("`itinerary_agent` successfully defined with `google_search` tool.")

`itinerary_agent` successfully defined with `google_search` tool.


In [26]:
# Run the Itinerary Agent
ITINERARY_USER_INPUT = "I'm visiting Barcelona for 3 days and I love art, food, and dance. Can you plan my trip?"
ITINERARY_SESSION_ID = f"{APP_NAME}_itinerary_session_001" # Unique session ID for this interaction

print("--- Running Itinerary Agent with Google Search ---")

# Invoke the itinerary agent using our modular helper function.
# This agent will internally decide to use the `google_search` tool based on its instructions.
itinerary_response = await run_adk_agent_interaction(
    agent=itinerary_agent,
    user_id=USER_ID,
    session_id=ITINERARY_SESSION_ID,
    input_text=ITINERARY_USER_INPUT,
    app_name=APP_NAME,
    session_service=common_session_service
)

# Display the agent's response, rendering the Markdown.
display(Markdown(f"{itinerary_response}"))

print("--- Itinerary Agent Interaction Complete ---")

--- Running Itinerary Agent with Google Search ---


Exception in thread Thread-12 (_asyncio_thread_main):
Traceback (most recent call last):
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 1043, in _bootstrap_inner
    self.run()
    ~~~~~~~~^^
  File "C:\Users\D5DR2B7\AppData\Roaming\Python\Python313\site-packages\ipykernel\ipkernel.py", line 772, in run_closure
    _threading_Thread_run(self)
    ~~~~~~~~~~~~~~~~~~~~~^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\threading.py", line 994, in run
    self._target(*self._args, **self._kwargs)
    ~~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\site-packages\google\adk\runners.py", line 165, in _asyncio_thread_main
    asyncio.run(_invoke_run_async())
    ~~~~~~~~~~~^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\D5DR2B7\AppData\Local\Programs\Python\Python313\Lib\asyncio\runners.py", line 195, in run
    return runner.run(main)
           ~~~~~~~~~~^^^^^^
  File "c:

No final response from agent.

--- Itinerary Agent Interaction Complete ---


In [27]:
# Congratulations 🎉 You’ve now successfully built and run two different LLM agents in ADK:
# A Packing List Agent for smart, personalized packing suggestions.
# An Itinerary Agent that effectively uses a google_search tool to plan an amazing trip with live, up-to-date recommendations.
# This demonstrates the power of ADK in combining LLM reasoning with external tools to create truly intelligent and practical applications. 
# This is the foundation for building even richer, multi-agent travel assistants and other sophisticated AI solutions with Google ADK.